In [1]:
import torch
import torch.nn as nn
import timm
import torchvision
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from types import MethodType

# --- Configuration ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = 'vit_base_patch16_224'
PATCH_SIZE = 32
BATCH_SIZE = 16 # Use a larger batch size for faster evaluation
TARGET_LABEL = 957 # The 'spaghetti' class we are targeting
NUM_EVAL_IMAGES = 1000 # How many images to test the patch on

print(f"Using device: {DEVICE}")

# --- 1. Load Model (No need for custom class for evaluation) ---
model = timm.create_model(MODEL_NAME, pretrained=True).to(DEVICE)
model.eval()

data_config = timm.data.resolve_model_data_config(model)
transform = timm.data.create_transform(**data_config, is_training=False)

# --- 2. Load the Optimized Patch and Test Data ---
try:
    adversarial_patch = torch.load("7's.pt", map_location=DEVICE)
    print("Successfully loaded '7's.pt'")
except FileNotFoundError:
    print("ERROR: Could not find '7's.pt'. Please run the training script first.")
    exit()

test_dataset = torchvision.datasets.Food101(root="./data", split='test', download=True, transform=transform)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
print(f"Loaded Food101 test set with {len(test_dataset)} images.")

# --- 3. Run The Evaluation Loop ---
successful_attacks = 0
total_images = 0

pbar = tqdm(test_dataloader, desc="Evaluating Patch")
for images, _ in pbar:
    if total_images >= NUM_EVAL_IMAGES:
        break
        
    images = images.to(DEVICE)
    
    # Apply the patch to each image in the batch at a random location
    patched_images = []
    img_height, img_width = data_config['input_size'][1:]
    for img in images:
        start_x = random.randint(0, img_width - PATCH_SIZE)
        start_y = random.randint(0, img_height - PATCH_SIZE)
        
        patched_img = img.clone()
        patched_img[:, start_y:start_y+PATCH_SIZE, start_x:start_x+PATCH_SIZE] = adversarial_patch
        patched_images.append(patched_img)
    
    patched_images = torch.stack(patched_images)

    # Get model predictions
    with torch.no_grad(): # No need to calculate gradients
        logits = model(patched_images)
    
    predictions = torch.argmax(logits, dim=1)
    
    # Check how many predictions match the target label
    successful_attacks += (predictions == TARGET_LABEL).sum().item()
    total_images += images.size(0)
    
    # Update progress bar
    current_asr = (successful_attacks / total_images) * 100
    pbar.set_postfix({"Success Rate": f"{current_asr:.2f}%"})


# --- 4. Report Final Results ---
final_asr = (successful_attacks / total_images) * 100
print("\n--- Evaluation Complete ---")
print(f"Tested patch on {total_images} images.")
print(f"Successful attacks (predicted class {TARGET_LABEL}): {successful_attacks}")
print(f"Attack Success Rate (ASR): {final_asr:.2f}%")
print("---------------------------")


Using device: cuda


C:\Users\DEVESH\AppData\Local\Temp\ipykernel_16740\1769582079.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adversarial_patch = torch.load("7's.pt", map_location=DEVI

Successfully loaded '7's.pt'
Loaded Food101 test set with 25250 images.


Evaluating Patch:   4%|▍         | 63/1579 [00:27<10:51,  2.33it/s, Success Rate=1.79%]


--- Evaluation Complete ---
Tested patch on 1008 images.
Successful attacks (predicted class 957): 18
Attack Success Rate (ASR): 1.79%
---------------------------
